In [5]:
import matplotlib.pyplot as plt
import math
import numpy as np
import pandas as pd

import csv
import os

In [7]:
folder = 'FDs'
files = [f for f in os.listdir(folder) if 'Lung' in f]

In [8]:
df_list = []
for f in files:
    df_dict = {}
    with open(os.path.join(folder, f), 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        csv_data = list(csv_reader)
    
    sc_arr = np.asarray(csv_data[4:]).astype(int)
    sizes, counts = sc_arr[:, 0], sc_arr[:, 1]
    
    df_dict = {
        'SID':                 '_'.join(f.split('_')[:-2]),
        'Lobe':                '',
        'Fractal_Dimension':   float(csv_data[0][1]),
        'Coefficients':        np.array([csv_data[1][1], csv_data[2][1]]),
        'Sizes':               sizes,
        'Counts':              counts,
    }
    
    if 'left' in f:
        df_dict['Lobe'] = 'left'
    elif 'right' in f:
        df_dict['Lobe'] = 'right'
    else:
        df_dict['Lobe'] = 'whole'
    
    df_list.append(df_dict)

df = pd.DataFrame(df_list)
display(df)

,SID,Lobe,Fractal_Dimension,Coefficients,Sizes,Counts
0,001_pect_phfirst_060413,left,2.313082,"[-2.3130816202789095, 13.895248532201418]","[128, 64, 32, 16, 8, 4, 2]","[15, 69, 330, 1821, 9867, 44330, 204723]"
1,001_pect_phfirst_060413,right,2.420838,"[-2.420837847458621, 14.496203971249454]","[128, 64, 32, 16, 8, 4, 2]","[16, 79, 421, 2487, 14778, 71185, 331731]"
2,001_pect_phfirst_060413,whole,2.368625,"[-2.368624972646306, 14.914710703893443]","[256, 128, 64, 32, 16, 8, 4, 2]","[6, 31, 153, 755, 4305, 24633, 115492, 536835]"
3,002_pect_phfirst_050913,left,2.327606,"[-2.327605909711189, 13.746505443552422]","[128, 64, 32, 16, 8, 4, 2]","[12, 55, 279, 1499, 8237, 37699, 173043]"
4,002_pect_phfirst_050913,right,2.311592,"[-2.3115915241510203, 13.952682016766452]","[128, 64, 32, 16, 8, 4, 2]","[16, 68, 367, 2043, 10819, 46750, 207703]"
...,...,...,...,...,...,...
664,387_pect_phfirst_090117,right,2.333411,"[-2.333410981147058, 13.781155612138587]","[128, 64, 32, 16, 8, 4, 2]","[12, 53, 283, 1590, 8992, 39753, 165099]"
665,387_pect_phfirst_090117,whole,2.352999,"[-2.352999263440592, 14.50372411321365]","[128, 64, 32, 16, 8, 4, 2]","[20, 112, 565, 3143, 17724, 78487, 328214]"
666,388_pect_phfirst_080516,left,2.302641,"[-2.3026406278494016, 12.579665812534031]","[64, 32, 16, 8, 4, 2]","[19, 100, 502, 2648, 12190, 54307]"
667,388_pect_phfirst_080516,right,2.281048,"[-2.2810482771023035, 12.836789208277484]","[128, 64, 32, 16, 8, 4, 2]","[7, 24, 127, 646, 3648, 16821, 74265]"


In [ ]:
# TODO:
# R2
# Residuals
# Local Slopes
# Figure out how to match different sizes (by lobes)
# Histogram of stuff

In [11]:
sizes_log = np.log(df['Sizes'][0])
counts_log = np.log(df['Counts'][0])

display(sizes_log, counts_log)

correlation_matrix = np.corrcoef(sizes_log, counts_log)
r_squared = correlation_matrix[0, 1]**2

display(correlation_matrix)
r_squared

array([4.85203026, 4.15888308, 3.4657359 , 2.77258872, 2.07944154,
       1.38629436, 0.69314718])

array([ 2.7080502 ,  4.2341065 ,  5.79909265,  7.50714108,  9.19695113,
       10.69941693, 12.22941312])

array([[ 1.        , -0.99981628],
       [-0.99981628,  1.        ]])

0.9996325907648529